In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re
# SQliteで作成
path = ''
#データベースの名前
db_names = 'google.スクレイピング.db'
#接続の確率
conn = sqlite3.connect(path + db_names)
c = conn.cursor()
#テーブルの作成
sql = '''
    CREATE TABLE IF NOT EXISTS repository (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT, 
        main_language TEXT, 
        stars INTEGER
    );
    '''
c.execute(sql)
conn.commit()

# スクレイピングを行う。
print("今からスクレイピングを始めるで")
print("実行時間が長い可能性があるので、気長に待ちます")
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}
pages = 1
total_count = 0

while True:
    URL = f"https://github.com/orgs/google/repositories?page={pages}"
    print(f'--- {pages}ページ目 取得中... (現在: {total_count}件) ---')
    try:
        res = requests.get(URL, headers=headers)
        
        if res.status_code != 200:
            print(f"アクセス失敗: {res.status_code}")
            break
            
        soup = BeautifulSoup(res.text, "html.parser")
        
        # リポジトリのリスト要素を取得
        repos = soup.find_all('li', class_=re.compile("Box-row|ListItem-module__listItem"))

        # 終了条件: リポジトリが見つからない場合
        if not repos:
            print("これ以上リポジトリが見つかりません。スクレイピングを終了します。")
            break

        for repo in repos:
            try:
                name_tag = repo.find('a', itemprop="name codeRepository")
                if not name_tag:
                    name_tag = repo.find('a', class_=re.compile("Title-module__anchor"))
                name = name_tag.text.strip() if name_tag else "Unknown"
                lang_tag = repo.find('span', itemprop="programmingLanguage")
                if not lang_tag:
                    lang_tag = repo.find(class_=re.compile("ReposListItem-module__Text|LanguageComponent"))
                language = lang_tag.text.strip() if lang_tag else "None"
                stars_count = 0
                star_tag = repo.select_one('a[href$="/stargazers"]')
                if star_tag:
                    stars_text = star_tag.get_text().strip()
                    stars_text = re.sub(r'\s+', '', stars_text)
                    try:
                        if 'k' in stars_text:
                            stars_count = int(float(stars_text.replace('k', '')) * 1000)
                        else:
                            stars_count = int(stars_text.replace(',', ''))
                    except ValueError:
                        stars_count = 0
                c.execute('INSERT INTO repository (name, main_language, stars) VALUES (?, ?, ?)', 
                          (name, language, stars_count))            
                total_count += 1
            except Exception as e:
                print(f"解析エラー(個別): {e}")
            time.sleep(1)            
        conn.commit()
        pages += 1
    except Exception as e:
        print(f'ページ処理エラー: {e}')
        break
# --- 3. 全データの表示 (クリーン版) ---
try:
    # 全件取得
    c.execute('SELECT * FROM repository')
    rows = c.fetchall()

    if not rows:
        print("データがありません。")
    else:
        # 簡潔にヘッダーを表示
        print("ID | Name | Language | Stars")
        
        for row in rows:
            # 取得したデータをそのまま表示
            print(f"{row[0]} | {row[1]} | {row[2]} | {row[3]}")
            
except sqlite3.Error as e:
    print("SELECT実行エラー:", e)

# --- 終了処理 ---
conn.close()
print('\nすべての処理が完了しました。')


    




今からスクレイピングを始めるで
実行時間が長い可能性があるので、気長に待ちます
--- 1ページ目 取得中... (現在: 0件) ---
--- 2ページ目 取得中... (現在: 30件) ---
--- 3ページ目 取得中... (現在: 60件) ---
--- 4ページ目 取得中... (現在: 90件) ---
--- 5ページ目 取得中... (現在: 120件) ---
--- 6ページ目 取得中... (現在: 150件) ---
--- 7ページ目 取得中... (現在: 180件) ---
--- 8ページ目 取得中... (現在: 210件) ---
--- 9ページ目 取得中... (現在: 240件) ---
--- 10ページ目 取得中... (現在: 270件) ---
--- 11ページ目 取得中... (現在: 300件) ---
--- 12ページ目 取得中... (現在: 330件) ---
--- 13ページ目 取得中... (現在: 360件) ---
--- 14ページ目 取得中... (現在: 390件) ---
--- 15ページ目 取得中... (現在: 420件) ---
--- 16ページ目 取得中... (現在: 450件) ---
--- 17ページ目 取得中... (現在: 480件) ---
--- 18ページ目 取得中... (現在: 510件) ---
--- 19ページ目 取得中... (現在: 540件) ---
--- 20ページ目 取得中... (現在: 570件) ---
--- 21ページ目 取得中... (現在: 600件) ---
--- 22ページ目 取得中... (現在: 630件) ---
--- 23ページ目 取得中... (現在: 660件) ---
--- 24ページ目 取得中... (現在: 690件) ---
--- 25ページ目 取得中... (現在: 720件) ---
--- 26ページ目 取得中... (現在: 750件) ---
--- 27ページ目 取得中... (現在: 780件) ---
--- 28ページ目 取得中... (現在: 810件) ---
--- 29ページ目 取得中... (現在: 840件) ---
--- 30ページ目 取得中...